In [ ]:
bds_state_sector_age = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_state_sector_age.csv')
bds_state_sector_age = bds_state_sector_age[bds_state_sector_age.year >=1986]
bds_state_sector_age.loc[bds_state_sector_age['firms'] == '(X)', 'firms'] = ''
bds_state_sector_age.loc[bds_state_sector_age['firms'] == '(D)', 'firms'] = 0

bds_state_sector_age.loc[bds_state_sector_age['emp'] == '(X)', 'emp'] = ''
bds_state_sector_age.loc[bds_state_sector_age['emp'] == '(D)', 'emp'] = 0

bds_state_sector_age.loc[bds_state_sector_age['firmdeath_firms'] == '(X)', 'firmdeath_firms'] = ''
bds_state_sector_age.loc[bds_state_sector_age['firmdeath_firms'] == '(D)', 'firmdeath_firms'] = 0
bds_state_sector_age['firms'] = pd.to_numeric(bds_state_sector_age['firms'], errors='coerce').astype('float')
bds_state_sector_age['emp'] = pd.to_numeric(bds_state_sector_age['emp'], errors='coerce').astype('float')
bds_state_sector_age['firmdeath_firms'] = pd.to_numeric(bds_state_sector_age['firmdeath_firms'], errors='coerce').astype('float')
bds_state_sector_age['death_rate'] = bds_state_sector_age['firmdeath_firms']/bds_state_sector_age['firms']
bds_state_sector_age = bds_state_sector_age[bds_state_sector_age['death_rate'] < 1]
bds_state_sector_age['entry'] = (bds_state_sector_age['fage'] == 'b) 1')
bds_state_sector_age['new'] = (bds_state_sector_age['fage'] == 'c) 2')|(bds_state_sector_age['fage'] == 'd) 3')|(bds_state_sector_age['fage'] == 'e) 4')|(bds_state_sector_age['fage'] == 'f) 5') 
bds_state_sector_age['young'] = (bds_state_sector_age['fage'] == 'g) 6 to 10')
bds_state_sector_age['old']  = (bds_state_sector_age['fage'] == 'h) 11 to 15') \
                               |(bds_state_sector_age['fage'] == 'i) 16 to 20') \
                               |(bds_state_sector_age['fage'] == 'j) 21 to 25') \
                               |(bds_state_sector_age['fage'] == 'k) 26+') \
                               |(bds_state_sector_age['fage'] == 'l) Left Censored') 
                               
bds_state_sector_age['age_gp'] = bds_state_sector_age['entry']* 1 + bds_state_sector_age['new']*2 + bds_state_sector_age['young'] *3 + bds_state_sector_age['old'] *4
bds_state_sector_age = bds_state_sector_age[bds_state_sector_age['age_gp'] != 0]
death_rate_all = pd.DataFrame()
death_rate_all['1985 - 1990'] = bds_state_sector_age[(bds_state_sector_age['year']>1985)&(bds_state_sector_age['year']<1991) ].groupby(['age_gp','sector'])['firmdeath_firms'].sum() / \
                            bds_state_sector_age[(bds_state_sector_age['year']>1985)&(bds_state_sector_age['year']<1991) ].groupby(['age_gp','sector'])['firms'].sum()

death_rate_all['2015 - 2019'] = bds_state_sector_age[(bds_state_sector_age['year']>2014)&(bds_state_sector_age['year']<2020) ].groupby(['age_gp','sector'])['firmdeath_firms'].sum() / \
                            bds_state_sector_age[(bds_state_sector_age['year']>2014)&(bds_state_sector_age['year']<2020) ].groupby(['age_gp','sector'])['firms'].sum()
death_rate_all['change_rate'] = - (death_rate_all['2015 - 2019'] - death_rate_all['1985 - 1990'])/death_rate_all['1985 - 1990']
death_rate_all = death_rate_all.reset_index()
mod = smf.ols(formula='change_rate ~  C(age_gp) + C(sector)', data=death_rate_all, missing='drop')
res = mod.fit()
res.summary()